<a href="https://colab.research.google.com/github/vincm1/RecSys_Implicit/blob/master/Bayesian_Personalized_Ranking_(BPR).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bayesian Personalized Ranking

This notebook will use the pairwise-ranking algorithm BPR to recommend and rank Top10 items based on the paper of Rendle et al. 2009 (https://arxiv.org/ftp/arxiv/papers/1205/1205.2618.pdf)

Therefore two different RecSys libraries will be used:



*   Implicit BPR(https://benfred.github.io/implicit/ by Ben Frederickson)
*   LightFM BPR(https://making.lyst.com/lightfm/docs/home.html by Maciej Kula)



In [1]:
import warnings
import zipfile
import time
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

import scipy.sparse as sparse

from datetime import datetime, timedelta
warnings.filterwarnings("ignore")

## Data Preprocessing

In [2]:
df_zip = zipfile.ZipFile('/content/drive/MyDrive/RecSys/Orders_Nov22_Jun23.zip')
df = pd.read_csv(df_zip.open('Bericht 1.csv'), delimiter=";")

In [3]:
df.head(2)

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,Product Descr2,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped
0,1547606,01.11.22,NaN,2022FM11,15515778,NET-S M. CHMIELEWSKI,Export Channel (DE),,9433B9X,INK CARTRIDGE SPS,BLACK 370ML 600 DPI INKJET BULK,1037,Consumables,Ink,Supplies,"-15,33",-1.0
1,1547615,01.11.22,NaN,2022FM11,15509465,DIGITAL RIVER IRELAND LIMITED,Export Channel (DE),DIRL,CB31510,LENOVO KEYBOARD PACK,FOR TAB P11-DE,641,Input Devices,Keyboards & Keypads,Printers & Peripherals,"-104,61",-1.0


In [4]:
print(len(df), len(df["BranchCustomerNbr"].unique()), len(df["Sku"].unique()))

2220299 17697 77401


In [5]:
# converting the customerid to string
df["BranchCustomerNbr"] = df["BranchCustomerNbr"].astype(str)
# converting the skuid to string
df["Sku"] = df["Sku"].astype(str)
# Entry Date to date
df['Entry Date'] = pd.to_datetime(df['Entry Date'], format='%d.%m.%y')
# dropping retours (orders with negative Qty shipped) and zero Qty shipped orders
df = df[df["Qty Shipped"] > 0]
# dropping backlog invoices, Specified date to filter the rows
specific_date = pd.to_datetime('2022-11-01')
# Filter the DataFrame to keep only the rows that are before or equal to the specific date
df = df[df["Entry Date"] >= specific_date]
#insert purchase indication column
df["purchase"] = 1

In [6]:
print(len(df), len(df["BranchCustomerNbr"].unique()), len(df["Sku"].unique()))

2028956 13894 75643


In [7]:
# add column that represent sku count
def sku_count(df_pl):

  df_pl['sku_count'] = df_pl.groupby('BranchCustomerNbr')['Sku'].transform('nunique')

  return df_pl

In [8]:
df = sku_count(df)

In [9]:
# drop customers that only purchased 1 SKU
df = df[df["sku_count"] > 1]

In [10]:
print(len(df), len(df["BranchCustomerNbr"].unique()), len(df["Sku"].unique()))

2025544 11328 75495


In [11]:
# Create a numeric user_id and artist_id column
df['BranchCustomerNbr'] = df['BranchCustomerNbr'].astype("category")
df['Sku'] = df['Sku'].astype("category")
df['bcn_id'] = df['BranchCustomerNbr'].cat.codes
df['sku_id'] = df['Sku'].cat.codes

In [12]:
df.head(2)

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,...,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped,purchase,sku_count,bcn_id,sku_id
213,1545306,2022-11-02,NaN,2022FM11,15885514,AXIS SOLUTION (PRIVATE) LIMITED,Export Channel (DE),,CF55877,DT PRINT ZQ320 KIT LABEL SENSOR,...,5805,AIDC/PoS Printers,Mobile Receipt Printer,"Other (incl. AIDC/POS, V7)","10.713,30",41.0,1,20,417,49730
458,4422886,2022-11-03,NaN,2023FM02,44413224,BWG INFORMATIONSYSTEME GMBH,Business Channel,,CF89211,Z-SELECT 2000D REMOVABLE NS,...,5812,AIDC/PoS Printers,Label Printers Supplies,"Other (incl. AIDC/POS, V7)","1.393,00",140.0,1,111,803,52649


In [13]:
users = df.bcn_id.unique()
items = df.sku_id.unique()
print(len(users), len(items))

11328 75495


In [14]:
df_2 = df[["bcn_id", "sku_id", "Entry Date", "Qty Shipped", "purchase"]]

In [15]:
grouped_df = df_2.groupby(["bcn_id", "sku_id"]).agg({
          "Qty Shipped":"sum",
          "purchase":"sum"}).reset_index()

In [16]:
# create binary column
grouped_df["purch_bin"] = 1

In [17]:
grouped_df.head(2)

,bcn_id,sku_id,Qty Shipped,purchase,purch_bin
0,0,925,1.0,1,1
1,0,3417,1.0,1,1


## Negative Sampling train

In [19]:
def negative_sampling(df_pl, bcn_ids, sku_ids, items, n_neg):
    """This function creates n_neg negative labels for every positive label

    @param user_ids: list of user ids
    @param sku_ids: list of sku ids
    @param items: unique list of sku ids
    @param n_neg: number of negative labels to sample

    @return df_neg: negative sample dataframe

    """

    neg = []
    ui_pairs = zip(bcn_ids, sku_ids)
    records = set(ui_pairs)

    # for every positive label case
    for (u, i) in records:
        # generate n_neg negative labels
        for _ in range(n_neg):
            j = np.random.choice(items)
            # resample if the movie already exists for that user
            while (u, j) in records:
                j = np.random.choice(items)
            neg.append([u, j, 0])

    # convert to pandas dataframe for concatenation later
    df_neg = pd.DataFrame(neg, columns=['bcn_id', 'sku_id', 'purchase'])

    #df_train = df_train[['bcn_id', 'sku_id']].assign(purchase=1)
    df_sampled = pd.concat([df_pl, df_neg], ignore_index=True)

    return df_sampled

## Implicit BPR

In [86]:
!pip install implicit

In [87]:
import implicit
from implicit.als import AlternatingLeastSquares as ALS
from implicit.bpr import BayesianPersonalizedRanking as BPR
from implicit.evaluation import leave_k_out_split, precision_at_k, mean_average_precision_at_k, ndcg_at_k, AUC_at_k, train_test_split
from sklearn.model_selection import GridSearchCV

In [88]:
from implicit.gpu.als import AlternatingLeastSquares as gpu_ALS
from implicit.gpu.bpr import BayesianPersonalizedRanking as gpu_BPR

In [89]:
csr_qty_matrix = sparse.csr_matrix((grouped_df['Qty Shipped'], (grouped_df['bcn_id'], grouped_df['sku_id'])))
csr_freq_matrix = sparse.csr_matrix((grouped_df['purchase'], (grouped_df['bcn_id'], grouped_df['sku_id'])))
csr_bin_matrix = sparse.csr_matrix((grouped_df['purch_bin'], (grouped_df['bcn_id'], grouped_df['sku_id'])))

In [90]:
csr_qty_matrix_train, csr_qty_matrix_test = leave_k_out_split(csr_qty_matrix, K=1, random_state=None)
csr_freq_matrix_train, csr_freq_matrix_test = leave_k_out_split(csr_freq_matrix, K=1, random_state=None)
csr_bin_matrix_train, csr_bin_matrix_test = leave_k_out_split(csr_bin_matrix, K=1, random_state=None)

### QTY Based BPR

In [52]:
auc10 = AUC_at_k(implicit_bpr_model, csr_freq_matrix_train, csr_qty_matrix_test, K=10)
prec10 = precision_at_k(implicit_bpr_model, csr_freq_matrix_train, csr_freq_matrix_test, K=10)
ndcg10 = ndcg_at_k(implicit_bpr_model, csr_freq_matrix_train, csr_freq_matrix_test, K=10)
print(f"AUC@10: {auc10}; PREC@10: {prec10}; NDCG@10: {ndcg10};; ")

  0%|          | 0/9656 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

AUC@10: 0.503248021295601; PREC@10: 0.09248135874067936; NDCG@10: 0.060152176712297785;; 


## Light FM BPR

In [19]:
!pip install lightfm

In [20]:
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k as lfm_precision_at_k, auc_score
from lightfm.cross_validation import random_train_test_split
import random

In [55]:
model = LightFM(learning_rate=0.05, loss='bpr')
model.fit(csr_qty_matrix_train, epochs=10)

In [58]:
train_precision = lfm_precision_at_k(model, csr_qty_matrix_train, k=10).mean()
test_precision = lfm_precision_at_k(model, csr_qty_matrix_test, k=10).mean()

train_auc = auc_score(model, csr_qty_matrix_train).mean()
test_auc = auc_score(model, csr_qty_matrix_test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.09, test 0.00.
AUC: train 0.79, test 0.75.


### Movielens Testing

In [21]:
from implicit.datasets.lastfm import get_lastfm

artists, users, artist_user_plays = get_lastfm()

In [22]:
print(len(artists), len(users))

292385 358868


In [42]:
user_plays_train, user_plays_test = train_test_split(user_plays, train_percentage=0.8)

In [43]:
implicit_gpu_bpr_model_movielens = gpu_BPR(factors=200, regularization=0.01, learning_rate=0.01)
implicit_gpu_bpr_model_movielens.fit(user_plays_train)

  0%|          | 0/100 [00:00<?, ?it/s]

In [45]:
implicit_gpu_als_model_movielens = gpu_ALS(factors=100, regularization=0.01, alpha=2.0)
implicit_gpu_als_model_movielens.fit(user_plays_train)

  0%|          | 0/15 [00:00<?, ?it/s]

In [47]:
auc10 = AUC_at_k(implicit_gpu_bpr_model_movielens, user_plays_train,user_plays_test, K=10)
prec10 = precision_at_k(implicit_gpu_bpr_model_movielens, user_plays_train, user_plays_test, K=10)
ndcg10 = ndcg_at_k(implicit_gpu_bpr_model_movielens, user_plays_train, user_plays_test, K=10)
print(f"AUC@10: {auc10}; PREC@10: {prec10}; NDCG@10: {ndcg10};; ")

  0%|          | 0/358496 [00:00<?, ?it/s]

  0%|          | 0/358496 [00:00<?, ?it/s]

  0%|          | 0/358496 [00:00<?, ?it/s]

AUC@10: 0.5282592030193779; PREC@10: 0.06346576148672824; NDCG@10: 0.06724882973989248;; 


In [48]:
auc10 = AUC_at_k(implicit_gpu_als_model_movielens, user_plays_train,user_plays_test, K=10)
prec10 = precision_at_k(implicit_gpu_als_model_movielens, user_plays_train, user_plays_test, K=10)
ndcg10 = ndcg_at_k(implicit_gpu_als_model_movielens, user_plays_train, user_plays_test, K=10)
print(f"AUC@10: {auc10}; PREC@10: {prec10}; NDCG@10: {ndcg10};; ")

  0%|          | 0/358496 [00:00<?, ?it/s]

  0%|          | 0/358496 [00:00<?, ?it/s]

  0%|          | 0/358496 [00:00<?, ?it/s]

AUC@10: 0.5635056968487524; PREC@10: 0.14258366476020595; NDCG@10: 0.14847250639584067;; 
